In [1]:
# load the games.csv

import pandas as pd
df =pd.read_csv('data/games.csv')
df_filter = df[['title', 'releaseDate', 'rating', 'platforms', 'genres', 'platform_metascores']]
print(df_filter.head(3))

print(df_filter.shape)


                                     title releaseDate rating  \
0  The Legend of Zelda: Breath of the Wild  2017-03-03   E10+   
1                          Baldur's Gate 3  2023-08-03      M   
2                               Elden Ring  2022-02-25      M   

                        platforms             genres platform_metascores  
0           Wii U,Nintendo Switch  Open-World Action               96,97  
1  PC,PlayStation 5,Xbox Series X        Western RPG            96,96,99  
2  PC,Xbox Series X,PlayStation 5         Action RPG            94,96,96  
(5638, 6)


In [2]:
# 首先查看原始数据的格式
print("原始数据：")
print(df_filter[['title', 'platforms', 'platform_metascores']].head())

# find the 3th title
print(df_filter.iloc[2]['title'])

# 修改分割函数
def split_multiple_platforms(df):
    """
    将包含多个平台的游戏记录拆分成多行
    """
    new_records = []
    
    for _, row in df.iterrows():
        platforms = row['platforms']
        scores = row['platform_metascores']
        
        # 检查并处理平台数据
        if pd.isna(platforms) or platforms == '[]':
            # 如果是空值或空列表，保持原样
            new_records.append(row)
        else:
            # 移除方括号并分割
            platforms_clean = platforms.strip('[]').replace("'", "").split(',')
            scores_clean = scores.strip('[]').replace("'", "").split(',') if not pd.isna(scores) else [None] * len(platforms_clean)
            
            # 为每个平台创建新记录
            for platform, score in zip(platforms_clean, scores_clean):
                new_record = row.copy()
                new_record['platforms'] = platform.strip()
                new_record['platform_metascores'] = score.strip() if score else None
                new_records.append(new_record)
    
    return pd.DataFrame(new_records)

# 使用修改后的函数处理数据
df_split = split_multiple_platforms(df_filter)

df_split = df_split[df_split['platforms'] == 'Nintendo Switch']

print("\n处理后的数据：")
print(df_split.shape)


原始数据：
                                               title  \
0            The Legend of Zelda: Breath of the Wild   
1                                    Baldur's Gate 3   
2                                         Elden Ring   
3  The House in Fata Morgana - Dreams of the Reve...   
4                                    Persona 5 Royal   

                                           platforms platform_metascores  
0                              Wii U,Nintendo Switch               96,97  
1                     PC,PlayStation 5,Xbox Series X            96,96,99  
2                     PC,Xbox Series X,PlayStation 5            94,96,96  
3                                    Nintendo Switch                  96  
4  PlayStation 4,Xbox Series X,PC,PlayStation 5,N...      95,94,95,91,94  
Elden Ring

处理后的数据：
(2308, 6)


In [3]:
# 檢查
print(df_split.head(4))


                                               title releaseDate rating  \
0            The Legend of Zelda: Breath of the Wild  2017-03-03   E10+   
3  The House in Fata Morgana - Dreams of the Reve...  2021-04-09      M   
4                                    Persona 5 Royal  2020-03-31      M   
6                       Portal: Companion Collection  2022-06-28      T   

         platforms             genres platform_metascores  
0  Nintendo Switch  Open-World Action                  97  
3  Nintendo Switch       Visual Novel                  96  
4  Nintendo Switch               JRPG                  94  
6  Nintendo Switch        Compilation                  95  


In [4]:
import pandas as pd
import numpy as np
df2 =pd.read_csv('data/games_reviews.csv')
# 應該還要加上 review_type ！辨識是 
df2 = df2[['title', 'date', 'score','platform', 'quote' , 'review_type']]


# 使用 numpy.where 來根據 review_type 調整 score
df2['score'] = np.where(df2['review_type'] == 'user', df2['score'] * 10, df2['score'])
    
#########################################################
df2 = df2[df2['platform'] == 'Nintendo Switch']
#########################################################
print(df2.head(10))
df2.shape

                          title        date  score         platform  \
0  Portal: Companion Collection  2023-11-23  100.0  Nintendo Switch   
1  Portal: Companion Collection  2024-09-13  100.0  Nintendo Switch   
2  Portal: Companion Collection  2024-03-08  100.0  Nintendo Switch   
3  Portal: Companion Collection  2023-11-30  100.0  Nintendo Switch   
4  Portal: Companion Collection  2024-08-08  100.0  Nintendo Switch   
5  Portal: Companion Collection  2024-07-12  100.0  Nintendo Switch   
6  Portal: Companion Collection  2024-11-17  100.0  Nintendo Switch   
7  Portal: Companion Collection  2025-04-18  100.0  Nintendo Switch   
8  Portal: Companion Collection  2024-09-03  100.0  Nintendo Switch   
9  Portal: Companion Collection  2025-01-03  100.0  Nintendo Switch   

                                               quote review_type  
0  Amazing, absolutely genius and brilliant from ...        user  
1  2 fantastic games in one amazing package. The ...        user  
2  theas to games

(103187, 6)

In [5]:
#df2[df2['quote'].duplicated()]

In [6]:
# 1 對多 (一個遊戲對應多個review)
df_merged = df_split.copy()  
# 如果需要重命名列
if 'platforms' in df_merged.columns:
    df_merged = df_merged.rename(columns={'platforms': 'platform'})


# 使用 merge 方法
merged_df = pd.merge(df2, df_merged,  on=['title', 'platform'], how ='left')
#print(merged_df)

# filter important colums
merged_df = merged_df[['title','platform', 'releaseDate', 'rating', 'genres', 'quote', 'score' ]]
#merged_df = merged_df.rename(columns={'title_x': 'title'})
merged_df

,title,platform,releaseDate,rating,genres,quote,score
0,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,"Amazing, absolutely genius and brilliant from ...",100.0
1,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,2 fantastic games in one amazing package. The ...,100.0
2,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,theas to games are amazing I have both of thea...,100.0
3,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,Portal companion cured my depression It ´ s ...,100.0
4,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,"As of typing out this review, I have finally c...",100.0
...,...,...,...,...,...,...,...
103182,Tiny Racer,Nintendo Switch,2020-07-31,E,Auto Racing,Tiny Racer is an extremely hard title to recom...,30.0
103183,Tiny Racer,Nintendo Switch,2020-07-31,E,Auto Racing,Tiny Racer offers nothing that you haven’t see...,20.0
103184,Tiny Racer,Nintendo Switch,2020-07-31,E,Auto Racing,Tiny Racer is one of the worst racing games av...,20.0
103185,Tiny Racer,Nintendo Switch,2020-07-31,E,Auto Racing,Tiny Racer may not be very ambitious but unfor...,20.0


In [7]:
#merged_df = merged_df[merged_df['platform']=='Nintendo Switch']
merged_df.shape
merged_df.to_csv('1st_website_data.csv')

In [ ]:
merged_df = merged_df[merged_df['platform']=='Nintendo Switch']
merged_df.shape


(103187, 7)

In [ ]:
merged_df.info()
merged_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103187 entries, 0 to 103186
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   title        103187 non-null  object 
 1   platform     103187 non-null  object 
 2   releaseDate  101101 non-null  object 
 3   rating       95035 non-null   object 
 4   genres       101101 non-null  object 
 5   quote        103179 non-null  object 
 6   score        102111 non-null  float64
dtypes: float64(1), object(6)
memory usage: 5.5+ MB


,title,platform,releaseDate,rating,genres,quote,score
0,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,"Amazing, absolutely genius and brilliant from ...",100.0
1,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,2 fantastic games in one amazing package. The ...,100.0
2,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,theas to games are amazing I have both of thea...,100.0
3,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,Portal companion cured my depression It ´ s ...,100.0
4,Portal: Companion Collection,Nintendo Switch,2022-06-28,T,Compilation,"As of typing out this review, I have finally c...",100.0


In [10]:
merged_df
# to check the merged_df['quote'] is duplicate
merged_df[merged_df['quote'].duplicated()]

,title,platform,releaseDate,rating,genres,quote,score
115,The House in Fata Morgana - Dreams of the Reve...,Nintendo Switch,2021-04-09,M,Visual Novel,Literally a perfect VN. Literally a perfect VN...,100.0
319,The House in Fata Morgana - Dreams of the Reve...,Nintendo Switch,2021-04-09,M,Visual Novel,"While it is weird to consider this game a ""gam...",100.0
320,The House in Fata Morgana - Dreams of the Reve...,Nintendo Switch,2021-04-09,M,Visual Novel,"Worth your time, one of the best stories you'l...",100.0
321,The House in Fata Morgana - Dreams of the Reve...,Nintendo Switch,2021-04-09,M,Visual Novel,"The art, the music, the atmosphere, all are su...",100.0
323,The House in Fata Morgana - Dreams of the Reve...,Nintendo Switch,2021-04-09,M,Visual Novel,The story in this way more mature than I expec...,100.0
...,...,...,...,...,...,...,...
102600,Double Dragon IV,Nintendo Switch,NaN,NaN,NaN,I really wanted to like Double Dragon IV. It's...,60.0
102601,Double Dragon IV,Nintendo Switch,NaN,NaN,NaN,"Double Dragon IV seems like a rushed project, ...",50.0
102625,Double Dragon IV,Nintendo Switch,NaN,NaN,NaN,"I enjoyed my time with Double Dragon 4, it loo...",75.0
102626,Double Dragon IV,Nintendo Switch,NaN,NaN,NaN,There's certainly nothing wrong with celebrati...,40.0


### to get the sales datas

In [ ]:
import pandas as pd
df3 =pd.read_csv('data/vgchartz-4_14_2025.csv')
print(df3.head(10))

df3.shape

# 取df3 title, total_sales 欄位
df3 = df3[['title', 'total_sales', 'na_sales',  'jp_sales', 'other_sales', 'pal_sales', 'release_date', 'console']]

# 找realse_date > 2015 
# 将release_date转换为datetime格式
df3['release_date'] = pd.to_datetime(df3['release_date'], errors='coerce')

# 找2015年之后的数据
filtered_df3 = df3[df3['release_date'].dt.year >= 2015]


# 去除空白字元後再判斷空字串
filtered_df3['total_sales'] = filtered_df3['total_sales'].astype(str).str.strip()
filtered_df3['total_sales'].replace('', pd.NA, inplace=True)



                                             img  \
0  /games/boxart/full_3351915AmericaFrontccc.jpg   
1  /games/boxart/full_6662824AmericaFrontccc.png   
2  /games/boxart/full_8122622AmericaFrontccc.jpg   
3  /games/boxart/full_6138740AmericaFrontccc.jpg   
4  /games/boxart/full_3094030AmericaFrontccc.jpg   
5  /games/boxart/full_6510540AmericaFrontccc.jpg   
6  /games/boxart/full_6800951AmericaFrontccc.jpg   
7  /games/boxart/full_5563178AmericaFrontccc.jpg   
8  /games/boxart/full_2804791AmericaFrontccc.jpg   
9                   /games/boxart/3887656ccc.jpg   

                               title console   genre  \
0                       Warriors      Series  Action   
1                  Devil May Cry      Series  Action   
2              God of War (2018)         All  Action   
3               Dynasty Warriors      Series  Action   
4              God of War (2018)         PS4  Action   
5             Grand Theft Auto V         PS3  Action   
6                        Frogger   

/var/folders/ym/cnwhpvr13b10x0g3j5y5kn9w0000gn/T/ipykernel_13213/2606277939.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df3['total_sales'] = filtered_df3['total_sales'].astype(str).str.strip()
/var/folders/ym/cnwhpvr13b10x0g3j5y5kn9w0000gn/T/ipykernel_13213/2606277939.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df3['total_sales'].replace('', pd.NA, inplace=True)


In [12]:




print("total_sales 欄位非空值筆數：", filtered_df3['total_sales'].notnull().sum())
print("total_sales 欄位為空值筆數：", filtered_df3['total_sales'].isnull().sum())

filtered_df3 = filtered_df3[filtered_df3['total_sales'].notnull()]


total_sales 欄位非空值筆數： 10098
total_sales 欄位為空值筆數： 0


In [13]:

#filtered_df3 = filtered_df3[filtered_df3['total_sales']=='nan']
filtered_df3 = filtered_df3[filtered_df3['console'].isin(['NS'])]
#filtered_df3 = filtered_df3[filtered_df3['console'].isin(['NS', 'PC'])]

In [14]:
filtered_df3

,title,total_sales,na_sales,jp_sales,other_sales,pal_sales,release_date,console
76,Hyrule Warriors: Age of Calamity,nan,NaN,NaN,NaN,NaN,2020-11-20,NS
205,Dragon Ball: Xenoverse 2,nan,NaN,NaN,NaN,NaN,2017-09-22,NS
239,Marvel Ultimate Alliance 3: The Black Order,nan,NaN,NaN,NaN,NaN,2019-07-19,NS
261,Resident Evil: Revelations Collection,nan,NaN,NaN,NaN,NaN,2017-11-28,NS
304,Astral Chain,nan,NaN,NaN,NaN,NaN,2019-08-30,NS
...,...,...,...,...,...,...,...,...
65835,VA-11 HALL-A,nan,NaN,NaN,NaN,NaN,2019-05-02,NS
65842,Vampire: The Masquerade – Shadows of New York,nan,NaN,NaN,NaN,NaN,2020-09-10,NS
65851,WILL: A Wonderful World,nan,NaN,NaN,NaN,NaN,2019-03-26,NS
65862,"Yoru, Tomosu",nan,NaN,NaN,NaN,NaN,2020-07-30,NS


In [15]:
# Step 1: 清空白與非標準 NaN 字串
filtered_df3['total_sales'] = (
    filtered_df3['total_sales']
    .astype(str)
    .str.strip()
    .replace(['', 'nan', 'NaN', 'None'], pd.NA)
)

# Step 2: 轉數值（強制非數字轉成 NaN）
filtered_df3['total_sales'] = pd.to_numeric(filtered_df3['total_sales'], errors='coerce')

# Step 3: 過濾非 NaN
filtered_df3 = filtered_df3[filtered_df3['total_sales'].notnull()]
filtered_df3

,title,total_sales,na_sales,jp_sales,other_sales,pal_sales,release_date,console
549,Monster Hunter Generations Ultimate,0.72,0.27,0.27,0.04,0.14,2018-08-28,NS
587,Diablo III: Eternal Collection,0.67,0.38,0.01,0.06,0.22,2018-11-02,NS
590,LEGO Marvel Super Heroes 2,0.67,0.31,0.01,0.06,0.29,2017-11-14,NS
616,LEGO The Incredibles,0.63,0.35,0.01,0.06,0.21,2018-06-15,NS
744,Fire Emblem Warriors,0.51,0.22,0.10,0.04,0.14,2017-10-20,NS
...,...,...,...,...,...,...,...,...
65519,Cendrillon palikA,0.00,NaN,0.00,NaN,NaN,2018-10-25,NS
65522,Hakuoki: Shinkai - Fuukaden,0.00,NaN,0.00,NaN,NaN,2018-09-06,NS
65528,Aokana: Four Rhythm Across the Blue,0.00,NaN,0.00,NaN,NaN,2020-08-21,NS
65533,"Nora, Princess, and Stray Cat",0.00,NaN,0.00,NaN,NaN,2018-10-25,NS


In [16]:
filtered_df3['console'].unique()

array(['NS'], dtype=object)

In [17]:

# df3 對應 merged_df (Cant not do that, since some title are not perfect match from 2 df)
'''
merged_2_df3 = pd.merge(merged_df, filtered_df3, on='title', how='left')
print(merged_2_df3.head(4))


merged_2_df3.info()


'''

"\nmerged_2_df3 = pd.merge(merged_df, filtered_df3, on='title', how='left')\nprint(merged_2_df3.head(4))\n\n\nmerged_2_df3.info()\n\n\n"

In [18]:
'''
print(merged_2_df3.head(4))
merged_2_df3.shape
'''

'\nprint(merged_2_df3.head(4))\nmerged_2_df3.shape\n'

In [19]:
#!pip install scikit-learn

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def similarity(dfa, dfb, reference_col='title', source_col='title',
               reference_value_cols=None, similarity_threshold=0.7):
    """
    將 dfb（來源資料）與 dfa（參考資料）中的 title 欄位做 fuzzy matching，
    並將 dfa 的額外欄位（如 total_sales 等）合併回 dfb。
    """
    # Step 1: 清理 title 欄位
    df_a = dfa[[reference_col]].dropna().drop_duplicates().reset_index(drop=True)
    df_b = dfb[[source_col]].dropna().drop_duplicates().reset_index(drop=True)

    # Step 2: 合併文字內容建 corpus
    corpus = pd.concat([df_a[reference_col], df_b[source_col]], ignore_index=True)

    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', lowercase=True)
    tfidf_matrix = vectorizer.fit_transform(corpus)

    tfidf_a = tfidf_matrix[:len(df_a)]
    tfidf_b = tfidf_matrix[len(df_a):]

    # Step 3: Cosine similarity
    cos_sim = cosine_similarity(tfidf_b, tfidf_a)

    # Step 4: 找最佳匹配
    best_match_indices = np.argmax(cos_sim, axis=1)
    best_match_scores = np.max(cos_sim, axis=1)
    matched_titles = df_a[reference_col].iloc[best_match_indices].values

    # Step 5: 套用閾值
    matched_titles = np.where(best_match_scores >= similarity_threshold, matched_titles, None)
    best_match_scores = np.where(best_match_scores >= similarity_threshold, best_match_scores, None)

    # Step 6: 將結果寫入 df_b
    df_b['matched_title'] = matched_titles
    df_b['similarity'] = best_match_scores

    # Step 7: 合併匹配結果回原始 dfb
    result = dfb.merge(df_b, on=source_col, how='left')

    # Step 8: 合併對應欄位資料（如 total_sales）
    if reference_value_cols:
        result = result.merge(
            dfa[[reference_col] + reference_value_cols],
            left_on='matched_title',
            right_on=reference_col,
            how='left',
            suffixes=('', '_ref')
        )

    print("✅ 成功匹配筆數：", result['matched_title'].notnull().sum())
    return result




In [ ]:
merged_sales  = similarity(
    dfa=filtered_df3,
    dfb=merged_df,
    reference_col='title',
    source_col='title',
    reference_value_cols=['total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales'],
    similarity_threshold=0.7
)

✅ 成功匹配筆數： 8717


In [22]:
merged_sales  =merged_sales [merged_sales ['matched_title'].notnull()]
merged_sales = merged_sales[['title' , 'platform','releaseDate' ,'rating', 'genres', 'quote', 'score','similarity', 'total_sales']]


In [23]:
merged_sales

,title,platform,releaseDate,rating,genres,quote,score,similarity,total_sales
8940,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Come on, it's Undertale! Anything below a 9 is...",100.0,1.0,0.02
8941,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Undertale is probably the best top down rpg, a...",100.0,1.0,0.02
8942,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"This game Is Perfect. I love the music, the ch...",100.0,1.0,0.02
8943,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,This game is a masterpiece. The dialogue is to...,100.0,1.0,0.02
8944,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Charming, exciting, thought-provoking, best OS...",100.0,1.0,0.02
...,...,...,...,...,...,...,...,...,...
103115,Shaq Fu: A Legend Reborn,Nintendo Switch,2018-06-05,T,2D Beat-'Em-Up,"Trying to play the nostalgia chord is risky, b...",45.0,1.0,0.02
103116,Shaq Fu: A Legend Reborn,Nintendo Switch,2018-06-05,T,2D Beat-'Em-Up,Its overly simplistic visuals are easy to forg...,40.0,1.0,0.02
103117,Shaq Fu: A Legend Reborn,Nintendo Switch,2018-06-05,T,2D Beat-'Em-Up,"""People crowdfunded this crap?"" Yes, they did....",30.0,1.0,0.02
103118,Shaq Fu: A Legend Reborn,Nintendo Switch,2018-06-05,T,2D Beat-'Em-Up,A bad experience in multiple ways. It seeks to...,30.0,1.0,0.02


In [24]:
merged_sales.to_csv('merged_sales_df.csv')

In [25]:
# to check the merged_sales['quote'] is duplicate
merged_sales[merged_sales['quote'].duplicated()]





,title,platform,releaseDate,rating,genres,quote,score,similarity,total_sales
12838,Dead Cells,Nintendo Switch,2018-08-07,T,Metroidvania,If you want to play a worthy successor to the ...,100.0,1.0,0.11
22564,Wolfenstein II: The New Colossus,Nintendo Switch,2017-10-27,M,FPS,Quotation forthcoming.,85.0,1.0,0.11
22680,Sonic Mania Plus,Nintendo Switch,2018-07-17,E,2D Platformer,The quality of the package in Mania Plus is so...,90.0,0.815005,0.47
24721,Sonic Mania,Nintendo Switch,2017-08-15,E,2D Platformer,Sonic Mania has quickly become my favorite Son...,100.0,1.0,0.47
24722,Sonic Mania,Nintendo Switch,2017-08-15,E,2D Platformer,Sonic has been a big part of my childhood and ...,100.0,1.0,0.47
...,...,...,...,...,...,...,...,...,...
100161,Sonic Forces,Nintendo Switch,2017-11-07,E10+,3D Platformer,Best Sonic game ever!\nThank you SEGA for crea...,100.0,1.0,0.74
100162,Sonic Forces,Nintendo Switch,2017-11-07,E10+,3D Platformer,Best Sonic game ever!\nThank you SEGA for crea...,100.0,1.0,0.74
100163,Sonic Forces,Nintendo Switch,2017-11-07,E10+,3D Platformer,Best Sonic game ever!\nThank you SEGA for crea...,100.0,1.0,0.74
100164,Sonic Forces,Nintendo Switch,2017-11-07,E10+,3D Platformer,Best Sonic game ever!\nThank you SEGA for crea...,100.0,1.0,0.74


#### get the price_data

In [26]:
df_price = pd.read_csv('data/dekudeals_games.csv')

df_price= df_price.rename(columns={    
    '遊戲名稱': 'title',
    '原價': 'original_price',
    '現價': 'current_price'

})
df_price = df_price[['title', 'original_price', 'current_price']]
df_price
#df_price.shape


,title,original_price,current_price
0,Golf Story,$14.99,$2.54
1,Spiritfarer,$29.99,$4.49
2,Burnout Paradise Remastered,$29.99,$4.79
3,Blasphemous,$24.99,$4.99
4,Ni no Kuni: Wrath of the White Witch,$49.99,$9.99
...,...,...,...
20368,未然探偵 The Protea Cases,NaN,NaN
20369,桃の華は鮮血に染まる,NaN,NaN
20370,猟奇リスナー ～ 狙われた姫配信者 ～ Lunatic Viewer - Streamer ...,NaN,NaN
20371,肉食ギャルは清純派！？ ～ Innocent Stuck-up Girls! ～,NaN,NaN


In [27]:
df_price = df_price[df_price['original_price'].notnull()]
df_price.shape
df_price

,title,original_price,current_price
0,Golf Story,$14.99,$2.54
1,Spiritfarer,$29.99,$4.49
2,Burnout Paradise Remastered,$29.99,$4.79
3,Blasphemous,$24.99,$4.99
4,Ni no Kuni: Wrath of the White Witch,$49.99,$9.99
...,...,...,...
13312,"Early Learning Games for Kids, Toddlers & Babies",$9.78,$8.80
13313,Halloween Games for Toddlers and Babies,$7.99,$7.19
13314,My Little Car Wash - Cars & Trucks Roleplaying...,$9.98,$8.98
13965,"Animal Farm Jigsaw Games for Toddlers, Babies ...",$9.78,$8.80


In [28]:
# to use similarity function join 'merged_df' and 'df_price'

merged_total_df = similarity(
    dfa = df_price, # dfa : ref
    dfb=merged_sales,  #dfb: source
    reference_col='title',
    source_col='title',
    reference_value_cols=['original_price'],
    similarity_threshold=0.4
)

✅ 成功匹配筆數： 6650


In [29]:
#merged_total_df
merged_total_df  =merged_total_df [merged_total_df ['matched_title'].notnull()]
merged_total_df

,title,platform,releaseDate,rating,genres,quote,score,similarity_x,total_sales,matched_title,similarity_y,title_ref,original_price
0,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Come on, it's Undertale! Anything below a 9 is...",100.0,1.0,0.02,Undertale,1.0,Undertale,$14.99
1,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Undertale is probably the best top down rpg, a...",100.0,1.0,0.02,Undertale,1.0,Undertale,$14.99
2,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"This game Is Perfect. I love the music, the ch...",100.0,1.0,0.02,Undertale,1.0,Undertale,$14.99
3,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,This game is a masterpiece. The dialogue is to...,100.0,1.0,0.02,Undertale,1.0,Undertale,$14.99
4,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Charming, exciting, thought-provoking, best OS...",100.0,1.0,0.02,Undertale,1.0,Undertale,$14.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8680,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,The Switch port has almost everything going wr...,40.0,1.0,0.24,Hello Neighbor,1.0,Hello Neighbor,$29.99
8681,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,"A confused, messy stealth-puzzler with flaky p...",40.0,1.0,0.24,Hello Neighbor,1.0,Hello Neighbor,$29.99
8682,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,You might be able to derive some enjoyment fro...,40.0,1.0,0.24,Hello Neighbor,1.0,Hello Neighbor,$29.99
8683,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,"Hello Neighbor could be fun, within the right ...",40.0,1.0,0.24,Hello Neighbor,1.0,Hello Neighbor,$29.99


In [30]:
clean_df = merged_total_df[['title', 'platform', 'releaseDate', 'rating', 'genres', 'quote', 'score', 'total_sales', 'original_price']]

clean_df

,title,platform,releaseDate,rating,genres,quote,score,total_sales,original_price
0,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Come on, it's Undertale! Anything below a 9 is...",100.0,0.02,$14.99
1,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Undertale is probably the best top down rpg, a...",100.0,0.02,$14.99
2,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"This game Is Perfect. I love the music, the ch...",100.0,0.02,$14.99
3,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,This game is a masterpiece. The dialogue is to...,100.0,0.02,$14.99
4,Undertale,Nintendo Switch,2015-09-15,NaN,JRPG,"Charming, exciting, thought-provoking, best OS...",100.0,0.02,$14.99
...,...,...,...,...,...,...,...,...,...
8680,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,The Switch port has almost everything going wr...,40.0,0.24,$29.99
8681,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,"A confused, messy stealth-puzzler with flaky p...",40.0,0.24,$29.99
8682,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,You might be able to derive some enjoyment fro...,40.0,0.24,$29.99
8683,Hello Neighbor,Nintendo Switch,2017-12-08,E10+,First-Person Adventure,"Hello Neighbor could be fun, within the right ...",40.0,0.24,$29.99


In [31]:
clean_df.to_csv('clean_df.csv')